In [41]:
import cv2
import numpy as np
from sklearn.linear_model import LinearRegression
import os


Використати стандартні методи лінійної алгебри для виконання цієї задачі через правильну роботу з векторами ознак і просту регресію або невеличку конволюційку, яку ви можете швидко зробити руками

In [42]:
directory = "test"

set_1_txt = []
set_1_jpg = []
# ≈ 150
set_2_txt = []
set_2_jpg = []
for name in os.listdir(directory):
    file_path = os.path.join(directory, name)
    if name.endswith('set2.txt'):
        set_2_txt.append(file_path)
    elif name.endswith('set2.jpg'):
        set_2_jpg.append(file_path)
    elif name.endswith('set1.txt'):
        set_1_txt.append(file_path)
    else:
        set_1_jpg.append(file_path)



In [43]:
def extract_features(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

    feature_vector = gradient_magnitude.flatten()

    return feature_vector


In [44]:
def load_label(label_path, image_shape):
    with open(label_path, 'r') as file:
        label_data = file.read().splitlines()
    
    labels = []
    for line in label_data:
        values = line.split()
        x_center, y_center, width, height = map(float, values[1:])
        x_min = (x_center - width / 2) * image_shape[1]
        y_min = (y_center - height / 2) * image_shape[0]
        x_max = (x_center + width / 2) * image_shape[1]
        y_max = (y_center + height / 2) * image_shape[0]
        labels.append((x_min, y_min, x_max, y_max))
    
    return labels

In [45]:

def crutch(all_features, all_labels):
    features = np.vstack(all_features)
    labels = np.vstack(all_labels)
    return features, labels

In [46]:
def regression(features, labels):
    model = LinearRegression()
    model.fit(features, labels)
    return model


In [47]:
def main(image_paths_train, coord_paths_train, image_paths_test, l=[]):
    all_features = []
    all_labels = []

    for image_path, label_path in zip(image_paths_train, coord_paths_train):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image_shape = image.shape

        features = extract_features(image)
        all_features.append(features)

        labels = load_label(label_path, image_shape)
        all_labels.append(labels)

    features, labels = crutch(all_features, all_labels)

    model = regression(features, labels)

    for image_path in image_paths_test:
        test_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        test_features = extract_features(test_image)
        test_features = np.array([test_features])
        predicted_label = model.predict(test_features)
        print(f"Predicted bounding box for {image_path}: {predicted_label}")

In [48]:
# image_paths = ['test/160_set2.jpg', 'test/195_set1.jpg', 'test/194_set1.jpg']
# label_paths = ['test/160_set2.txt', 'test/195_set1.txt', 'test/194_set1.txt']
image_paths = set_2_jpg[:10]
coord_paths = set_2_txt[:10]
image_paths_test = ['test/153_set1.jpg', 'test/212_set1.jpg', 'test/279_set1.jpg']
# main(set_2_jpg, set_2_txt, set_1_jpg, set_1_txt)
main(image_paths, coord_paths, image_paths_test, [])



Predicted bounding box for test/153_set1.jpg: [[316.1543712  490.57663496 541.35991314 731.25419499]]
Predicted bounding box for test/212_set1.jpg: [[287.97026012 487.344209   501.14420202 718.60321304]]
Predicted bounding box for test/279_set1.jpg: [[ 713.29216152  794.85774955 1031.11703454 1129.23742481]]
